# <p style="text-align: center;">Construction of an RNA-based Knowledge Graph</p>

***
***

**Authors:** [E. Cavalleri](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=emanuele.cavalleri@studenti.unimi.it), [T.J. Callahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com), [M. Mesiti](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=marco.mesiti@unimi.it), [G. Valentini](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=giorgio.valentini@unimi.it)

**GitHub Repositories:** [RNA-KG](https://github.com/AnacletoLAB/RNA-KG), [PheKnowLator](https://github.com/callahantiff/PheKnowLator/)  
<!--- **Release:** **[v2.0.0](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)** --->
  
<br>  

**Objective:** Knowledge Graphs (KGs) provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. 
[PheKnowLator](https://github.com/callahantiff/PheKnowLator) is a system that support the user in the acquisition of biomedical entities from different kinds of data sources and their representation in terms of a biomedical knowledge graph that models unbiased molecular mechanisms as prescribed by domain ontologies. In this notebook we wish to show the use of PheKnowLator for the generation of a RNA-based KG.

***
***

## Notebook Purpose
**Wiki Page:** **[`Current PheKnowLator release`](https://github.com/callahantiff/PheKnowLator/wiki/)**

<br>

**Purpose:** This notebook serves as a `main` file for the RNA-KG construction based on PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. Please see the [PheKnowLator README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**
1. Make sure that the following input documents have been constructed (see the [PheKnowLator Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/AnacletoLAB/RNA-KG/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/AnacletoLAB/RNA-KG/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/AnacletoLAB/RNA-KG/blob/main/resources/edge_source_list.txt)   

2. Download [`RELATIONS_LABELS.txt`](https://github.com/AnacletoLAB/RNA-KG/blob/main/resources/relations_data/RELATIONS_LABELS.txt) relations label file prior to running the scripts. Please see [PheKnowLator Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information on how it is generated.

3. Select a knowledge graph construction method (i.e. `instance-based` or `subclass-based`).  

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [1]:
# import needed libraries
import glob
import json
import pandas
import ray
import time

# import module
from pkt_kg.downloads import OntData, LinkedData
from pkt_kg.edge_list import CreatesEdgeList
from pkt_kg.knowledge_graph import FullBuild, PartialBuild, PostClosureBuild

***
## Download Data Sources <a class="anchor" id="data-source"></a>

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
  - [`resource_info.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/edge_source_list.txt)

<br>

**Assumption:** All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated prior to this step. For assistance with creating these datasets, see the [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/notebooks/RNA-KG_Preparation.ipynb) Jupyter Notebook.

***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator.

In [2]:
ont = OntData('resources/ontology_source_list.txt', 'resources/resource_info.txt')

ont.parses_resource_file()

In [3]:
ont.data_files = ont.source_list
ont.generates_source_metadata()


*** Generating Metadata ***



100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 60349.70it/s]


In [4]:
ont._writes_source_metadata_locally()

100%|██████████████████████████████████████████| 6/6 [00:00<00:00, 29296.65it/s]


In [5]:
ont.resource_info

['miRNA-gene|;;|entity-entity|RO_0011002|https://www.mirbase.org/cgi-bin/mature.pl?mature_acc=|http://www.ncbi.nlm.nih.gov/gene/|t|2;0|0:./resources/processed_data/MIRNA_MIRBASE_MAP.txt;1:./resources/processed_data/ENSEMBL_GENE_ENTREZ_GENE_MAP.txt|None|None',
 'miRNA-disease|;;|entity-class|RO_0003302|https://www.mirbase.org/cgi-bin/mature.pl?mature_acc=|http://purl.obolibrary.org/obo/|t|2;1|0:./resources/processed_data/MIRNA_MIRBASE_MAP.txt;1:./resources/processed_data/DOID_MONDO_MAP.txt|None|None',
 'gene-disease|;;|entity-class|RO_0003302|http://www.ncbi.nlm.nih.gov/gene/|http://purl.obolibrary.org/obo/|t|0;4|1:./resources/processed_data/DISEASE_MONDO_MAP.txt|10;>=;1.0|None',
 'mRNAv-disease|;;|entity-class|RO_0002606|https://go.drugbank.com/drugs/|http://purl.obolibrary.org/obo/|t|2;3|None|None|None',
 'miRNA-tsRNA|;;|entity-entity|RO_0002434|https://www.mirbase.org/cgi-bin/mature.pl?mature_acc=|https://rna.sysu.edu.cn/tsRFun/searchDetail-tsRNA.php?tsRNAid=|t|0;1|0:./resources/proc

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [6]:
edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')

edges.parses_resource_file()

In [7]:
edges.data_files = edges.source_list
edges.generates_source_metadata()


*** Generating Metadata ***



100%|████████████████████████████████████████| 56/56 [00:00<00:00, 59149.09it/s]


In [8]:
edges._writes_source_metadata_locally()

100%|████████████████████████████████████████| 56/56 [00:00<00:00, 49292.97it/s]


In [9]:
edges.source_list.keys()

dict_keys(['miRNA-gene', 'miRNA-disease', 'gene-disease', 'miRNA-tsRNA', 'tsRNA-disease', 'gene-snoRNA', 'snoRNA-miRNA', 'snoRNA-snoRNA', 'snoRNA-lncRNA', 'snoRNA-snRNA', 'lncRNA-disease', 'circRNA-disease', 'lncRNA-protein', 'protein-miRNA', 'chemical-miRNA', 'gRNA-gene', 'chemical-disease', 'ASO-gene', 'lncRNA-gene', 'chemical-gene', 'gene-gene', 'protein-protein', 'gene-protein', 'chemical-protein', 'siRNA-gene', 'siRNA-geneICBP', 'shRNA-gene', 'aptamer-protein', 'ASO-protein11007', 'ASO-protein10002', 'aptamer-disease', 'mRNAv-disease', 'ASO-disease', 'siRNA-disease', 'riboswitch-protein', 'riboswitch-gobp', 'ribozyme-gobp', 'viralRNA-ribozyme', 'chemical-pathway', 'chemical-gobp', 'gene-pathway', 'protein-pathway', 'protein-gobp', 'pathway-gocc', 'lncRNA-role', 'lncRNA-gocc', 'lncRNA-gobp', 'protein-gocc1018', 'miRNA-gobp', 'circRNA-miRNA', 'snoRNA-gobp', 'mRNA-gocc', 'miRNA-gocc', 'circRNA-gocc', 'chemical-gocc', 'protein-gocc1025'])

***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/main/notebooks/RNA-KG_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** `Master_Edge_List_Dict.json`. Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'miRNA-disease'  :
                {'source_labels'    : ';;',
                 'data_type'        : 'entity-class',
                 'edge_relation'    : 'RO_0003302',
                 'uri'              : ('https://www.mirbase.org/cgi-bin/mature.pl?mature_acc=',
                                       'http://purl.obolibrary.org/obo/'),
                 'delimiter'        : 't',
                 'column_idx'       : '2;1',
                 'identifier_maps'  : '0:./[...]/MIRNA_MIRBASE_MAP.txt;1:./[...]/DOID_MONDO_MAP.txt',
                 'evidence_criteria': 'None',
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [10]:
import psutil
# set-up environment for parallel processing -- even if running program serially these steps are needed
cpus = psutil.cpu_count(logical=True)
ray.init()

2023-05-19 08:23:07,331	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.4.0


In [11]:
# combine data sources
combined_edges = dict(edges.data_files, **ont.data_files)
resource_info_loc = './resources/resource_info.txt'

# initialize edge dictionary class
master_edges = CreatesEdgeList(data_files=combined_edges, source_file=resource_info_loc)
master_edges.runs_creates_knowledge_graph_edges(source_file=resource_info_loc, data_files=combined_edges, cpus=cpus)

(CreatesEdgeList pid=2267737) Finished Edge: snoRNA-snRNA (snoRNA = 31, snRNA = 11); 45 unique edges
(CreatesEdgeList pid=2267741) Finished Edge: circRNA-disease (circRNA = 28, disease = 10); 29 unique edges
(CreatesEdgeList pid=2267737) Finished Edge: siRNA-geneICBP (siRNA = 76, geneICBP = 53); 78 unique edges
(CreatesEdgeList pid=2267734) Finished Edge: snoRNA-snoRNA (snoRNA = 86, snoRNA = 74); 344 unique edges
(CreatesEdgeList pid=2267727) Finished Edge: miRNA-disease (miRNA = 308, disease = 91); 1656 unique edges
(CreatesEdgeList pid=2267741) Finished Edge: aptamer-protein (aptamer = 2, protein = 4); 4 unique edges
(CreatesEdgeList pid=2267735) Finished Edge: snoRNA-lncRNA (snoRNA = 15, lncRNA = 11); 17 unique edges
(CreatesEdgeList pid=2267746) Finished Edge: gRNA-gene (gRNA = 77, gene = 43); 77 unique edges
(CreatesEdgeList pid=2267735) Finished Edge: siRNA-gene (siRNA = 4, gene = 4); 4 unique edges
(CreatesEdgeList pid=2267727) Finished Edge: ASO-gene (ASO = 7, gene = 5); 7 uniq

**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [12]:
master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))
master_edges.keys()

dict_keys(['mRNAv-disease', 'gRNA-gene', 'protein-gocc1018', 'chemical-miRNA', 'aptamer-disease', 'lncRNA-gobp', 'protein-miRNA', 'ASO-protein10002', 'lncRNA-gocc', 'lncRNA-protein', 'ASO-protein11007', 'lncRNA-role', 'circRNA-disease', 'aptamer-protein', 'pathway-gocc', 'lncRNA-disease', 'shRNA-gene', 'protein-gobp', 'snoRNA-snRNA', 'siRNA-geneICBP', 'protein-pathway', 'snoRNA-lncRNA', 'siRNA-gene', 'gene-pathway', 'snoRNA-snoRNA', 'chemical-protein', 'chemical-gobp', 'protein-gocc1025', 'snoRNA-miRNA', 'gene-protein', 'chemical-pathway', 'chemical-gocc', 'gene-snoRNA', 'protein-protein', 'viralRNA-ribozyme', 'circRNA-gocc', 'tsRNA-disease', 'gene-gene', 'ribozyme-gobp', 'miRNA-gocc', 'miRNA-tsRNA', 'chemical-gene', 'riboswitch-gobp', 'mRNA-gocc', 'gene-disease', 'lncRNA-gene', 'riboswitch-protein', 'snoRNA-gobp', 'miRNA-disease', 'ASO-gene', 'siRNA-disease', 'circRNA-miRNA', 'miRNA-gene', 'chemical-disease', 'ASO-disease', 'miRNA-gobp'])

In [13]:
# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}

# function to return key for any value
def get_key(my_dict, val):
    for key, value in my_dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"

# print basic stats on each resource
edge_data = [[key,
              get_key(relation_dict, 'http://purl.obolibrary.org/obo/'+master_edges[key]['edge_relation']),
              ', '.join(master_edges[key]['edge_list'][0]),
              len(master_edges[key]['edge_list'])]
             for key in master_edges.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df                

,Edge Type,Relation,Example Edge,Unique Edges
0,mRNAv-disease,is substance that treats,"DB15695, MONDO_0100096",7
1,gRNA-gene,decreases by repression quantity of,"www.addgene.org/74705, 60",77
2,protein-gocc1018,contained in,"PR_P11310, GO_1990742",18746
3,chemical-miRNA,regulates activity of,"CHEBI_63791, MIMAT0004488",1746
4,aptamer-disease,is substance that treats,"DB04998, MONDO_0004992",4
5,lncRNA-gobp,participates in,"84557, GO_0006914",99
6,protein-miRNA,involved in regulation of,"PR_P35869, MI0000734",1930
7,ASO-protein10002,is carrier of,"DB16699, PR_P02768",1
8,lncRNA-gocc,contained in,"7105, GO_0005829",33858
9,lncRNA-protein,interacts with,"79854, PR_Q969S3",2735


<br><br>

***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)** 

**Jupyter Notebooks:**  
- [`RNA-KG_Preparation.ipynb`](https://github.com/emanuelecavalleri/testRNA-KG/blob/master/notebooks/RNA-KG_Preparation.ipynb)  
[`Ontology_Cleaning.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb)  


<br>

**Assumptions:**  
- <u>Construction Approach</u>. If using the `subclass-based` construction approach, please make sure that a `pickled` dictionary mapping each non-ontology data node to an existing ontology class is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/tree/master/resources/knowledge_graphs#construction-method) for additional information).   
- <u>Relations Data</u>. If inverse relation data is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/relations_data/README.md) for additional information).  
- <u>Node Metadata</u>. If node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/node_metadata` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/node_data/README.md) for additional information).  
- <u>Decoding OWL Semantics</u>. If decoding OWL-Semantics, please make sure to provide a list of owl:Property types to keep is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information). 

<br>

**Input:** 
- `Master_Edge_List_Dict.json`  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph` (`.owl` and Networkx MultiDiGraph `.pkl`)  
- `Class Instance URI-UUID Map` (if "instance" construction approach)   
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies. See [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontologies/README.md) for additional information on how to preprocess the ontologies prior to merging them.    

2. Create Edges. Add edge lists to merged ontologies.  

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Filter OWL Semantics. Filter the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information).

5. Save Edge Lists and Node Metadata. Several versions of the knowledge graph are saved, including: the full knowledge graph (`owl` or Networkx MultiDiGraph `pickle`), triple lists (i.e. integer index and identifier labeled edge lists with a dictionary that maps between the integer indices and node identifiers), and a file of metadata (i.e. identifiers, labels, synonyms, and descriptions) for all nodes in the knowledge graph.  

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 10 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.

***


In [ ]:
# specify input arguments
build = 'full'
construction_approach = 'instance'
add_node_data_to_kg = 'no'
add_inverse_relations_to_kg = 'no'
decode_owl_semantics = 'yes'
kg_directory_location = './resources/knowledge_graphs'

In [ ]:
# construct knowledge graphs
if build == 'partial':
    kg = PartialBuild(construction=construction_approach,
                      node_data=add_node_data_to_kg,
                      inverse_relations=add_inverse_relations_to_kg,
                      decode_owl=decode_owl_semantics,
                      cpus=cpus,
                      write_location=kg_directory_location)
elif build == 'post-closure':
    kg = PostClosureBuild(construction=construction_approach,
                          node_data=add_node_data_to_kg,
                          inverse_relations=add_inverse_relations_to_kg,
                          decode_owl=decode_owl_semantics,
                          cpus=cpus,
                          write_location=kg_directory_location)
else:
    kg = FullBuild(construction=construction_approach,
                   node_data=add_node_data_to_kg,
                   inverse_relations=add_inverse_relations_to_kg,
                   decode_owl=decode_owl_semantics,
                   cpus=cpus,
                   write_location=kg_directory_location)

kg.construct_knowledge_graph()
ray.shutdown()